<a href="https://colab.research.google.com/github/marimiya/hello-world/blob/master/Copy_of_ONT_pyCUDA_Align.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this exercise we will be creating a python code to generate some artificial DNA sequences that will be our set of reference genes. We will then create a new sequence that is similar to one of the references and create a function that finds the best match. We will measure the performance of this code and then consider how we might create a GPU-accelerated version.

First of all let's import a few libraries

In [0]:
import random
import numpy as np
import time

Next, set up some constants to define the size of the problem

In [0]:
REFERENCE_LENGTH = 500  # The length of the reference sequences
SEQUENCE_LENGTH = 100    # The length of the sequence to be generated (before noise is added)
NUMBER_REFERENCES = 100  # The number of references in which to find a match

# Dictionary that converts the base letters to an integer
ascii_to_index = {"C": 0, "G": 1, "A": 2, "T": 3}
# Array that can be used to get the base letter from the integer value
index_to_ascii = ["C", "G", "A", "T"]

Create a function to create a populate some artificial reference sequences, which we will be searching for the best alignment

In [0]:
# Randomly assign CAGTs to the reference sequences
def initialise_references():
    refs = []

    for i in range(NUMBER_REFERENCES):
        ref = ""
        for j in range(REFERENCE_LENGTH):
            irand = random.randint(0, 3)
            ref += index_to_ascii[irand]

        refs.append(ref)

    return refs


Now we are going to generate the sequence to match by randomly selecting one of the reference sequences and then adding some noise to it.

In [0]:
# Randomly pick one of the references, pick an offset and
# add noise - this will be the sequence we'll seek
def initialise_sequence(references):

    seq = ""
    # randomly select the reference string to match
    iref = random.randint(0, NUMBER_REFERENCES-1)
    ref_offset = random.randint(0,REFERENCE_LENGTH - SEQUENCE_LENGTH)
    print("Actual offset = " + str(ref_offset) + " on sequence #" + str(iref))
    ref = references[iref]
    dels = 0
    subs = 0
    ins = 0

    for i in range(ref_offset, ref_offset+SEQUENCE_LENGTH):
        base = ref[i]
        i_rand = random.randint(0, 1000)

        if (i_rand < 22):
            # Insertion of up to 4 characters
            i_len = random.randint(0, 3)
            ins += i_len
            cnew=""
            for i in range(i_len):
                irand = random.randrange(0, 3)
                cnew += index_to_ascii[irand]

            seq+=cnew + base
        elif (i_rand < 44):
            subs += 1
            # Substitute the current character
            inew = random.randint(0, 2)
            if (base == "A"):
                cnew = index_to_ascii[inew if (inew == 2) else 3]
            elif (base == "T"):
                cnew = index_to_ascii[inew]
            elif (base == "C"):
                cnew = index_to_ascii[inew+1]
            else:
                cnew = index_to_ascii[inew if (inew == 1) else 0]

            seq+=cnew
        elif (i_rand < 66):
            # Deletion
            dels += 1
        else:
            seq += base

    print(str(subs) + " subs, " + str(dels) + " dels," + str(ins) + " ins")
    print(ref)
    print(seq)

    return seq


Next, create a cost function. In this case it is simple +1 for a match and -1 for anything else (insertion, deletion or substitution)

In [0]:
def cost(char1, char2):
  return -1 if char1!=char2 else +1

Next we create the main alignment function

In [0]:
# Compute a cost and, optionally, an offset for the semi-global alignment of seq to ref
def align(seq, ref, matrix, trace, compute_trace=False):

    max_score = -len(seq)
    max_idx = 0

    # Initialise the scoring matrices
    for i in range(len(seq)):
        matrix[i] = -(i+1)
        if compute_trace:
            trace[i, 0] = -i

    # Do the NW matrix calculation (column by column - to preserve memory)
    for i, base_r in enumerate(ref):
        top_left = 0
        bottom_left = matrix[0]
        top_right = 0
        for j, base_s in enumerate(seq):
            cost_m = cost(base_s, base_r) + top_left
            cost_i = bottom_left - 1
            cost_d = top_right - 1
            top_left = matrix[j]
            matrix[j] = max(cost_m, cost_i, cost_d)
            bottom_left = matrix[j+1]
            top_right = matrix[j]

            if compute_trace:
                if cost_m >= cost_i:
                    if cost_m >= cost_d:
                        trace[i + 1, j + 1] = [i, j]
                    else:
                        trace[i + 1, j + 1] = [i + 1, j]
                else:
                    if cost_i >= cost_d:
                        trace[i + 1, j + 1] = [i, j + 1]
                    else:
                        trace[i + 1, j + 1] = [i + 1, j]

        if matrix[len(seq)-1] > max_score:
            max_score = matrix[len(seq)-1]
            max_idx = i

    return max_score, trace, max_idx


We also need a function to compute the cost of the aligment using the back-tracking part of the Needleman-Wunsch algorithm

In [0]:
# Use the traceback to get the offset
def compute_offset(traceback, len_seq, ref, max_idx):

    match = ""
    idx = max_idx
    ls = len_seq

    # Traceback to get the offset
    while ls > 0:
        match = ref[idx - 1] + match
        idx, ls = traceback[idx, ls]
        #print(idx, ls)

    return idx



Right, now we have all the main functions defined we can create the main body of the program that will execute the functions we defined in order to find the best alignment between our references and the noisy sequence.

In [8]:
offset = 0
score = 0
max_score = 0
ref_idx = 0
time_span = 0

# For degugging, it can be helpful to set the seed so that the same strings are produced each time
#random.seed(30)

t1 = time.time()

refs = initialise_references()
seq = initialise_sequence(refs)

# Create arrays for the scoring matrix and traceback matrix
score_matrix = np.zeros(len(seq) + 1, dtype=np.int32)
trace_matrix = np.zeros((REFERENCE_LENGTH+1, len(seq) + 1, 2), dtype=np.int32)
max_score = -len(seq)

time_span = time.time() - t1
print("Data initialisation took " + str(time_span) + " milliseconds.")

# Run alignment on the host
t1 = time.time()

scores=[]
for i in range(NUMBER_REFERENCES):
    score, _, _ = align(seq, refs[i], score_matrix, trace_matrix)
    scores.append(score)
    if score > max_score:
        max_score = score
        ref_idx = i

# now get offset
score, trace_matrix, offset_idx = align(seq, refs[ref_idx], score_matrix, trace_matrix, compute_trace=True)
offset = compute_offset(trace_matrix, len(seq), refs[ref_idx], offset_idx)
print("Optimal cost of " + str(max_score) + " found at offset " + str(offset) + " in reference " + str(ref_idx))
time_span = time.time() - t1
print("Serial took " + str(time_span) + " seconds")

Actual offset = 12 on sequence #33
4 subs, 2 dels,3 ins
GGAGGGCGCTCGCCATCGGATGGTACGGTCGTGACTGACTACCACTAATTATTTTAAGGGGTAGAAGGTAACCACGGGCGTGCCTGACAGCTGTCACGAAACCCTATGAGTTTTAGGCGGTGCGATGGTTTTCCAGTGGATTTCCGACCGCTGGGAACACTACACTCCTAGTCAACTCTTGTTTGGGTAGGCCGTTAATAGTTCTACGAGCGTTCAAAGATAACTCACTGAACGAAGGAATTCGCCCAGTGCACTTTACATCAGAATTGGGCTACAACTCTTCGCTTGTACTTTGGGGGGTCTAGATAGGGGTTGAGGAAATAACGCGAGGGCGACGTATACTTTGGATTTAAACCTTAAGAATTTTCATCAGCACAAACCAACGTCCCTCATCCTCATCGATGATTGTCGTCAACACGCGCGGTTCTTTCTGTCTCAGGTCGCGCACATATGGTTGCGGGTACGCAGGGTGGGAGTGAAGCTGCTTGGCATCTGCACCA
CCATCGGATGGTACGCTCGTGACTGACTAGCAGACTATTTATTTTAAGGGGTAGAAGGTAACCACGGGCGTGCCTGACAGCTGTCACGAAACCCTATAGTT
Data initialisation took 0.0770113468170166 milliseconds.
Optimal cost of 90 found at offset 12 in reference 33
Serial took 38.44259071350098 seconds


Great - now can you create a kernel to do the alignment? Have a think for 10 minutes before you start implementing anything.

We also need to install pycuda and to import the libraries (for this, insert a scratch code window and type 'pip install pycuda')


In [90]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray
import numpy as np

random.seed(30)

refs = initialise_references()
seq = initialise_sequence(refs) 

seq_byte = bytearray(seq,'utf8')
seq_array = np.array(seq_byte)
max_score_array = np.array(np.int(max_score))
refs2 = [bytearray(i,'utf8') for i in refs]

full_marix = np.zeros(len(seq)*NUMBER_REFERENCES, dtype=np.int32)

gpu_refs = gpuarray.to_gpu(refs2)
gpu_seq = gpuarray.to_gpu(seq)
gpu_matrix = gpuarray.to_gpu(fullmatrix)
gpu_scores = gpuarray.to_gpu(max_score_array)


# Create your kernel (or kernels)
myfunc = SourceModule("""
__global__ void aligngpu(char* seq, int* len_seq, char* refs, int num_ref, int num_refs, int* matrix, int* max_score)
{
    int idx = thredIDx.x + blockDim.x*blockDim.x;
    int gridSize = gridDim.x * blockDim.x;   

    for (int  )

    # Initialise the scoring matrices
    for (int j=0;j < len_seq;j++)
        matrix[offset + j] = -(j+1);

    # Do the NW matrix calculation (column by column - to preserve memory)
    for (int i=0;i<0;i<len_ref;i++)
    {
      char base_r = refs[offset + i];
      int top_left = 0;
      int botom_left = matrrix[offset];
      int top_right = 0;
      
      for (int j=0;j < len_seq;j++)
      {
        char base_s = seq[j];
        int cost_m = (base_s == base_r ? 1 : -1) + top_left;




      }




    }







}
""")

# Create an empty 1d array
d = np.zeros((16), dtype=np.uint32)
# Allocate GPU memory
d_gpu = cuda.mem_alloc(d.nbytes)
# clear the gpu memory
cuda.memcpy_htod(d_gpu, d)

# Get the function handle
func = myfunc.get_function("aligngpu")
len_seq = np.int32(len(seq))
len_ref = np.int32(REFERENCE_LENGTH)
num_ref = np.int32(NUMBER_REFERENCES)
maxidx = np.int32(0)

func = gpu_seq.gpudata, len_seq, gpu_refs.gpudata, len_ref, num_ref, gpu_matrix.gpudata, gpu_scores.gpudata, block = (64,1,1), grid - (64,1,1))

max_s = np.int32(-len_seq)
scores = gpu_scores.get()
max_idx = 0


for i in range(NUMBER_REFERENCES);
  if scores[i] > max_s:
    max_s = scores[i]
    max_idx = i

score, trace_matrix, offset_idx = align(seq, refs[max_idx], score_matrix, trace_matrix, compute_trace=True)

offset = compute_offset(trace_matrix, len(seq), refs[max_idx], offset_idx)
print("Optimal cost of " + str(max_score) + " found at offset " + str(offset) + " in reference " + str(ref_idx))
time_span = time.time() - t1
print("CUDA version took " + str(time_span) + " seconds")

Actual offset = 299 on sequence #3
3 subs, 5 dels,2 ins
TCCCTCAAGGTTAGGGGCGTCCCATTACGTCCCTGCACGGGCTTAGTGTAGATTAAATCGTAGACGCTTACTTCGCCCTGATTGCACAGCCGTAGACCCCCTAAGTGTGCGGGGATTCGAGATTTGGTAGGTCCTCTATGAGAGTCAAACCGGAGAGGTCTTTGCGAGCTACGGTAACTTGCACGACTAGGACAACTCCCGGTCCCTAGCCACAACCCTGCCCAAGAGACTTAAGGTCCGGATGCCTATGATTCGGAGTAGTGCTCCAACTGCCGAAAGCCTGCTATGAGCCGTCCTCGTGAGAAGCACTGCCATCTTCGTGTTACGTGCGGAGTTGGTAATTAACTATGGTCCCTCTTGAGCTACTGGCCTTGAGAAGCTTATCTCTGGCCTGATCGTACAGAGACCACGGAATCCGTCAGTCCCACGCGGGTCTTACTACCCATCAGGTGCACCATTTTAGACTGTGTAACTCACGAGTCAAAGAGTCGCCAGAACTC
TGAGAAGCACTGCATCTTCGTGTTACGTGACCGGATTGGTAATTATCTATGGTACCTTTGAGCTACTGCCTTGAGAAGCTTATCTCTGGCCGATCGT


AttributeError: ignored

In [0]:
## GPU version of align function
# Compute a cost and, optionally, an offset for the semi-global alignment of seq to ref
def aligngpu(seq, ref, matrix, trace, compute_trace=False):

    max_score = -len(seq)
    max_idx = 0

    # Initialise the scoring matrices
    for i in range(len(seq)):
        matrix[i] = -(i+1)
        if compute_trace:
            trace[i, 0] = -i

    # Do the NW matrix calculation (column by column - to preserve memory)
    for i, base_r in enumerate(ref):
        top_left = 0
        bottom_left = matrix[0]
        top_right = 0
        for j, base_s in enumerate(seq):
            cost_m = cost(base_s, base_r) + top_left
            cost_i = bottom_left - 1
            cost_d = top_right - 1
            top_left = matrix[j]
            matrix[j] = max(cost_m, cost_i, cost_d)
            bottom_left = matrix[j+1]
            top_right = matrix[j]

            if compute_trace:
                if cost_m >= cost_i:
                    if cost_m >= cost_d:
                        trace[i + 1, j + 1] = [i, j]
                    else:
                        trace[i + 1, j + 1] = [i + 1, j]
                else:
                    if cost_i >= cost_d:
                        trace[i + 1, j + 1] = [i, j + 1]
                    else:
                        trace[i + 1, j + 1] = [i + 1, j]

        if matrix[len(seq)-1] > max_score:
            max_score = matrix[len(seq)-1]
            max_idx = i

    return max_score, trace, max_idx


In [75]:
new_refs = list()
new_ref = []
refnum = len(refs)

for j in range(refnum-1):
  ref_ind = list()
  print(j)
  for i in list(refs[j]):
    ref_ind.append(ascii_to_index.get(i))
    new_ref = ''.join([str(m) for m in ref_ind])
    new_refs[j] = new_ref

0


IndexError: ignored

In [60]:
refone = refs[0]
ref_ind = []

for i in list(refone):
  ref_ind.append(ascii_to_index.get(i))

print(ref_ind)

new_ref = []
new_ref = ''.join([str(i) for i in ref_ind])

print(new_ref)


[3, 3, 0, 1, 1, 0, 2, 3, 0, 2, 1, 2, 1, 3, 2, 2, 3, 3, 1, 1, 3, 3, 0, 3, 0, 0, 3, 3, 2, 3, 2, 0, 2, 1, 2, 2, 3, 3, 1, 2, 0, 1, 1, 1, 1, 2, 1, 1, 3, 2, 2, 3, 1, 0, 1, 0, 3, 3, 2, 0, 0, 0, 1, 2, 3, 2, 2, 3, 3, 1, 2, 0, 0, 2, 1, 2, 0, 3, 0, 1, 3, 0, 1, 0, 3, 3, 2, 3, 3, 1, 0, 0, 0, 3, 2, 0, 0, 3, 1, 3, 1, 2, 2, 1, 2, 1, 3, 3, 0, 3, 2, 0, 2, 0, 2, 0, 3, 0, 3, 3, 1, 0, 3, 3, 2, 0, 1, 2, 1, 0, 0, 3, 0, 1, 0, 0, 2, 3, 0, 0, 2, 2, 2, 0, 3, 2, 0, 1, 1, 3, 2, 1, 1, 0, 1, 0, 2, 0, 0, 0, 2, 2, 0, 2, 3, 0, 1, 1, 1, 2, 2, 1, 2, 2, 3, 3, 3, 3, 0, 1, 3, 2, 0, 2, 3, 0, 1, 1, 3, 1, 1, 3, 3, 1, 3, 1, 1, 3, 1, 1, 0, 2, 0, 2, 0, 1, 2, 3, 0, 0, 3, 2, 3, 0, 1, 3, 1, 3, 1, 3, 0, 2, 1, 0, 2, 1, 2, 0, 1, 2, 0, 0, 2, 1, 3, 0, 3, 2, 2, 1, 2, 1, 3, 1, 2, 2, 1, 1, 3, 1, 0, 1, 0, 2, 0, 2, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 1, 1, 2, 0, 0, 3, 0, 0, 3, 0, 2, 1, 3, 2, 3, 1, 3, 0, 0, 2, 0, 1, 1, 2, 0, 0, 2, 3, 2, 3, 1, 2, 3, 1, 2, 0, 2, 0, 0, 1, 2, 2, 1, 3, 3, 2, 1, 3, 1, 1, 3, 3, 3, 0, 3, 1, 0, 3, 0, 2, 3, 1, 0, 2, 3, 3, 3, 

In [70]:
refs2 = np.empty((NUMBER_REFERENCES,REFERENCE_LENGTH),dtype=bytes)
print(refs2.shape)


(100, 500)


In [74]:
for ref, i in enumerate(refs, refs2):
  #print(i)
  print(ref) 
  #refs2[i] = ascii_to_index.get(j)

TypeError: ignored

In [83]:

seq_byte = bytearray(seq,'utf8')
seq_array = np.array(seq_byte)
max_score_array = np.array(np.int(max_score))


90


Now create the calling code in Python (you can leave all the other functions as they were)

In [87]:
# Now run the same using our kernel
t1 = time.time()

#TODO - Insert data preparation code

random.seed(30)

refs = initialise_references()
seq = initialise_sequence(refs) 

seq_byte = bytearray(seq,'utf8')
seq_array = np.array(seq_byte)
max_score_array = np.array(np.int(max_score))
refs2 = [np.array(i,'utf8') for i in refs]

time_span = time.time() - t1
print("Data initialisation took " + str(time_span) + " milliseconds.")

t1 = time.time()

#TODO Insert alignment code

max_score, trace, max_idx = align(seq, refs2, matrix, trace, compute_trace=False)


# now get offset on the best reference match
score, trace_matrix, offset_idx = align(seq, refs[max_idx], score_matrix, trace_matrix, compute_trace=True)

offset = compute_offset(trace_matrix, len(seq), refs[max_idx], offset_idx)
print("Optimal cost of " + str(max_s) + " found at offset " + str(offset) + " in reference " + str(ref_idx))
time_span = time.time() - t1
print("CUDA version took " + str(time_span) + " seconds")

Actual offset = 299 on sequence #3
3 subs, 5 dels,2 ins
TCCCTCAAGGTTAGGGGCGTCCCATTACGTCCCTGCACGGGCTTAGTGTAGATTAAATCGTAGACGCTTACTTCGCCCTGATTGCACAGCCGTAGACCCCCTAAGTGTGCGGGGATTCGAGATTTGGTAGGTCCTCTATGAGAGTCAAACCGGAGAGGTCTTTGCGAGCTACGGTAACTTGCACGACTAGGACAACTCCCGGTCCCTAGCCACAACCCTGCCCAAGAGACTTAAGGTCCGGATGCCTATGATTCGGAGTAGTGCTCCAACTGCCGAAAGCCTGCTATGAGCCGTCCTCGTGAGAAGCACTGCCATCTTCGTGTTACGTGCGGAGTTGGTAATTAACTATGGTCCCTCTTGAGCTACTGGCCTTGAGAAGCTTATCTCTGGCCTGATCGTACAGAGACCACGGAATCCGTCAGTCCCACGCGGGTCTTACTACCCATCAGGTGCACCATTTTAGACTGTGTAACTCACGAGTCAAAGAGTCGCCAGAACTC
TGAGAAGCACTGCATCTTCGTGTTACGTGACCGGATTGGTAATTATCTATGGTACCTTTGAGCTACTGCCTTGAGAAGCTTATCTCTGGCCGATCGT


TypeError: ignored